In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

from layers import DenseLayer
from cnn import NeuralNetwork

X = np.reshape([[0, 0], [0, 1], [1, 0], [1, 1]], (4, 2, 1))
Y = np.reshape([[0], [1], [1], [0]], (4, 1, 1))

layers = [
    DenseLayer((2,), 3, activation="tanh"),
    DenseLayer((3,), 1, activation="tanh"),
]

# train
model = NeuralNetwork()
model.layers = layers
model.train(X, Y, validation=False, epochs=1000, learning_rate=0.1, batch_size=1)

# decision boundary plot
points = []
for x in np.linspace(0, 1, 20):
    for y in np.linspace(0, 1, 20):
        z = model.predict(np.reshape([[x], [y]], (2, 1)))
        points.append([x, y, z[0,0]])

points = np.array(points)

# fig = plt.figure()
# ax = fig.add_subplot(111, projection="3d")
# ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=points[:, 2], cmap="winter")
# plt.show()

In [2]:
from keras.datasets import mnist
from keras import utils

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

from layers import ConvLayer, FlattenLayer, DenseLayer, MaxPoolLayer, MeanPoolLayer
from losses import *
from cnn import NeuralNetwork

def preprocess_data(x, y, limit):
    zero_index = np.where(y == 0)[0][:limit]
    one_index = np.where(y == 1)[0][:limit]
    two_index = np.where(y == 2)[0][:limit]
    three_index = np.where(y == 3)[0][:limit]
    four_index = np.where(y == 4)[0][:limit]
    # print(zero_index.shape, one_index.shape)
    all_indices = np.hstack((zero_index, one_index, two_index, three_index, four_index))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    print(y.shape)
    y = utils.to_categorical(y)
    print(y.shape)
    y = y.reshape(len(y), 5, 1)
    return x, y

# load MNIST from server, limit to 100 images per class since we're not training on GPU
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(y_test)
x_train, y_train = preprocess_data(x_train, y_train, 1000)
x_test, y_test = preprocess_data(x_test, y_test, 1000)

# neural network
layers = [
    ConvLayer(16, 3, (1, 28, 28), activation="sigmoid"),
    MeanPoolLayer((16,26,26), pool_size=(2,2)),
    ConvLayer(16, 5, (16, 13, 13), activation="sigmoid"),
    MaxPoolLayer((16,9,9), pool_size=(3,3)),
    FlattenLayer((16, 3, 3)),
    DenseLayer((144,), 750, activation="sigmoid"),
    DenseLayer((750,), 5, activation="sigmoid"),
]

layer2s = [
    ConvLayer(5, 3, (1, 28, 28), activation="sigmoid"),
    FlattenLayer((5, 26, 26)),
    DenseLayer((5 * 26 * 26,), 100, activation='sigmoid'),
    DenseLayer((100,), 3, activation="sigmoid"),
]


model = NeuralNetwork(loss='binary_crossentropy')
model.layers = layers
# for i in range(10):
#     print(x_test[i], y_test[i])
# train
model.train(
    x_train,
    y_train,
    epochs=50,
    learning_rate=0.01,
    batch_size=16,
    optimizer="adam",
)

# test
# for x, y in zip(x_test, y_test):
#     output = model.predict(x)
#     print(f"pred: {np.argmax(output)}, true: {np.argmax(y)}")

In [8]:
model.validate(x_test, y_test)  

(0.6271664651350262,
 array([[[0.13083727],
         [0.0256694 ],
         [0.05443572],
         [0.21277194],
         [0.25116711]],
 
        [[0.24993288],
         [0.00166179],
         [0.11880071],
         [0.50965827],
         [0.03017573]],
 
        [[0.00883617],
         [0.00536   ],
         [0.31663769],
         [0.44036233],
         [0.03450001]],
 
        ...,
 
        [[0.04045882],
         [0.01319302],
         [0.02352264],
         [0.24366313],
         [0.11193275]],
 
        [[0.00850237],
         [0.11119629],
         [0.09568476],
         [0.16043337],
         [0.10025858]],
 
        [[0.00310316],
         [0.09389137],
         [0.02372895],
         [0.0602503 ],
         [0.50393932]]]))